In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import numpy as np
from re import sub, split, M 
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from matplotlib import pyplot as plt

In [ ]:
noSentences = 0
corpus=[]
visited_links=[]
file=open('text.txt', 'w')
def scrape():
    global corpus
    global visited_links
    global noSentences
    req = requests.get(queue.pop(0))
    html_content = req.content
    soup = BeautifulSoup(html_content, 'html5lib')
    news_corpus = soup.find_all('a')
    for i in soup.find_all("section", class_="entry-content"):
        list = i.findChildren("p")
        for x in list:
            sent_list=x.get_text()
            sentence = nltk.sent_tokenize(sent_list)
            corpus = corpus + sentence
            noSentences = noSentences + len(sentence)
            #print(noSentences)
            if(noSentences >= 10500):
                return
    for i in news_corpus:
        link=i.get('href')
        if(link is None):
            continue
        if((link.startswith("https://longreads.com/")) and (link not in visited_links)):
            queue.append(link)
            visited_links.append(link)
            #print(link)


url = 'https://longreads.com/'
queue = [url]
req = requests.get(url)
while(len(queue)>0 and noSentences<10000):
    scrape()
for i in corpus: 
    print(i)
    file.write(i)
print(noSentences)
file.close()

In [ ]:
# Word tokenization
tokens=[]
file=open('tokens.txt','w')
banned = ['.',',','|','***','?','!','\\','\/',':', ')', '(', '"','—',';', '...', '“', '”', '’']
for current in corpus:
    current = current.strip()
    current = sub(r'([.!?])', r'\1 ', current)
    current = split(r'  ',current)
    # if not current[-1]:
    #     del(current[-1])
    for i in current:
        i = nltk.word_tokenize(i)
        for x in i:
            if(x not in banned):
                tokens.append(x)
                file.write(f'{x}\n')
print(tokens)
file.close()

In [ ]:
#POS Tagging
file=open('postags.txt','w')
pos_tags = nltk.pos_tag(tokens)
for i in pos_tags:
    print(f"{i[0]} --> {i[1]}\n")
    file.write(f"{i[0]} --> {i[1]}\n")
file.close()

In [ ]:
#Remove stopwords
tokens_without_sw = [word for word in tokens if not word.lower() in stopwords.words('english')]
file=open("nostop.txt", "w")
for i in tokens_without_sw:
    print(f"{i}\n")
    file.write(f"{i}\n")
file.close()

In [ ]:
#Stemming
stemmed_tokens = []
file=open('stems.txt','w')
for i in tokens_without_sw:
    stemmed_tokens.append(nltk.PorterStemmer().stem(i))
    file.write(f'{nltk.PorterStemmer().stem(i)}\n')
print(stemmed_tokens)
file.close()

In [ ]:
#Lemmatizing
lemmatized_tokens = []
file=open('lemmas.txt','w')
for i in tokens_without_sw:
    lemmatized_tokens.append(nltk.WordNetLemmatizer().lemmatize(i))
    file.write(f'{nltk.WordNetLemmatizer().lemmatize(i)}\n')
print(lemmatized_tokens)
file.close()

###Token frequency

In [ ]:
token_counts = Counter(tokens_without_sw)
token_counts = [[word, freq] for word,freq in sorted(token_counts.items(), key=lambda i: i[1])][::-1]

print(token_counts)

courses = [token_counts[i][0] for i in range(0, 22)]
frequencies = [token_counts[i][1] for i in range(0,22)]
print(courses)
print(frequencies)
fig = plt.figure(figsize=(22,10))
plt.xlabel("Tokens")
plt.ylabel("No. of times it appears")
plt.title("Token Frequency")
plt.bar(courses, frequencies)
plt.plot(courses,frequencies)
plt.show()

###POS Tag Frequency

In [ ]:
taglist=[]
for line in pos_tags:
    taglist.append(line[1])

postag_counts = Counter(taglist)
postag_counts = [[word, freq] for word,freq in sorted(postag_counts.items(), key=lambda i: i[1])][::-1]

print(postag_counts)
courses = [postag_counts[i][0] for i in range(0, 10)]
frequencies = [postag_counts[i][1] for i in range(0,10)]
print(courses)
print(frequencies)
fig = plt.figure(figsize=(11,10))
plt.xlabel("POS Tags")
plt.ylabel("No. of times it appears")
plt.title("POS Tag Frequency")
plt.bar(courses, frequencies)
plt.plot(courses,frequencies)
plt.show()

###Stem frequency

In [ ]:
stem_counts = Counter(stemmed_tokens)
stem_counts = [[word, freq] for word,freq in sorted(stem_counts.items(), key=lambda i: i[1])][::-1]

print(stem_counts)

courses = [stem_counts[i][0] for i in range(0, 10)]
frequencies = [stem_counts[i][1] for i in range(0,10)]
print(courses)
print(frequencies)
fig = plt.figure(figsize=(10,10))
plt.xlabel("Stems")
plt.ylabel("No. of times it appears")
plt.title("Stem Frequency")
plt.bar(courses, frequencies)
plt.plot(courses,frequencies)
plt.show()

###Lemma frequency

In [ ]:
lemma_counts = Counter(lemmatized_tokens)

lemma_counts = [[word, freq] for word,freq in sorted(lemma_counts.items(), key=lambda i: i[1])][::-1]

print(lemma_counts)

courses = [lemma_counts[i][0] for i in range(0, 10)]
frequencies = [lemma_counts[i][1] for i in range(0,10)]
print(courses)
print(frequencies)
fig = plt.figure(figsize=(10,10))
plt.xlabel("Lemmas")
plt.ylabel("No. of times it appears")
plt.title("Lemma Frequency")
plt.bar(courses, frequencies)
plt.plot(courses,frequencies)
plt.show()

#Word Cloud

In [ ]:
token_counts = Counter(tokens_without_sw)
token_counts = [[word, freq] for word,freq in sorted(token_counts.items(), key=lambda i: i[1])][::-1]
# file=open('nostop.txt','r')
# data=file.read()
courses = [token_counts[i][0] for i in range(0, 21)]
print(courses)
wc = WordCloud(background_color = 'black', width = 1920, height = 1080)
wc.generate_from_text(" ".join(courses))
wc.to_file('wordcloud.png')
# file.close()